In [ ]:
import dask
import odc.geo.xr  # noqa: F401
from datacube import Datacube
from distributed import Client
from xarray import merge

In [ ]:
from odc.stac import configure_s3_access

configure_s3_access(aws_unsigned=True, cloud_defaults=True)

In [ ]:
# Configure Dask
# dask.config.set(
#     {
#         "distributed.dashboard.link": "https://hub.csiro.easi-eo.solutions/{JUPYTERHUB_SERVICE_PREFIX}proxy/{port}/status"
#     }
# )

client = Client()
client

In [ ]:
# Study site in Tasmania
bbox = [146.2357, -43.6796, 147.147,-42.9305]

# All of tasmania
bbox = [144.0, -43.6, 148.0, -40.0]

lon = (bbox[0], bbox[2])
lat = (bbox[1], bbox[3])

year = "2017"
product = "s1_gamma0_geotif_scene"
resolution = 100

dc = Datacube()

In [ ]:
data = dc.load(
    product=product,
    lon=lon,
    lat=lat,
    time=year,
    dask_chunks=dict(x=2024, y=2024),
    output_crs="epsg:3577",
    resolution=(-resolution, resolution),
)
data

In [ ]:
data.isel(time=1).vv.plot.imshow(robust=True)

In [ ]:
masked = data.where(data != data.vv.nodata)
masked.isel(time=1).vv.plot.imshow(robust=True)

In [ ]:
arrays = []
for band in ["vv", "vh"]:
#     arrays.append(data[band].median("time").rename(f"median_{band}"))
    arrays.append(data[band].mean("time").rename(f"mean_{band}"))
#     arrays.append(data[band].std("time").rename(f"std_{band}"))

# # Add count
arrays.append(data["vv"].count("time").rename("count").astype("int16"))

# Merge the arrays together into a Dataset with the names we want
mosaic = merge(arrays, compat="override")

mosaic

In [ ]:
mosaic_computed = mosaic.compute()

In [ ]:
mosaic_computed["mean_vv"].odc.explore(robust=True)

In [ ]:
mosaic_computed.mean_vv.odc.write_cog("s1_mosaic_2017_vv.tif")

In [ ]:
mosaic_computed.mean_vh.odc.write_cog("s1_mosaic_2017_vh.tif")